In [ ]:
import cx_Oracle
import datastory
import pandas as pd
import numpy as np
import sys
import os
import importlib
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

from datetime import datetime
from google.cloud import secretmanager

sys.path.append('../lib')
import pandas_utils

importlib.reload(pandas_utils)

In [ ]:
import dataverk_tools.themes.navdesign as theme
import dataverk_tools.themes.navdesign.plotly_template as template
pio.templates.default = template

In [ ]:
secrets = secretmanager.SecretManagerServiceClient()
resource_name = f"projects/knada-gcp/secrets/vebjorn-rekkebo-bac1/versions/latest"
secret = secrets.access_secret_version(name=resource_name)
secrets = dict([line.split(":") for line in secret.payload.data.decode('UTF-8').splitlines()])

ORACLE_HOST = '10.53.136.15'
ORACLE_PORT = '1521'
ORACLE_SERVICE = 'pen'

In [ ]:
dsnStr = cx_Oracle.makedsn(ORACLE_HOST, ORACLE_PORT, service_name=ORACLE_SERVICE)

# Antall krav, førstegangsbehandling innland/utland

In [ ]:
con = cx_Oracle.connect(user=secrets["PEN_USER"], password=secrets["PEN_PASSWORD"],
                               dsn=dsnStr)

df_krav = pandas_utils.pandas_from_sql('../sql/kravcount.sql', con)
con.close()

In [ ]:
df = df_krav.copy()

In [ ]:
df.head()

In [ ]:
from google.cloud.bigquery import Client, LoadJobConfig

table_id = f'pensjon-saksbehandli-prod-1f83.saksstatistikk.krav_forstegang'
job_config = LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
)

client = Client(project="pensjon-saksbehandli-prod-1f83")

job = client.load_table_from_dataframe(df, table_id, job_config=job_config)
job.result()

In [ ]:
con = cx_Oracle.connect(user=secrets["PEN_USER"], password=secrets["PEN_PASSWORD"],
                               dsn=dsnStr)

df_tmp = pandas_utils.pandas_from_sql('../sql/tmp.sql', con)
con.close()

In [ ]:
df_tmp.sort_values("ANTALL", ascending=False).reset_index(drop=True).to_csv("k_krav_gjelder i pesys.csv")

# Ledetid

In [ ]:
con = cx_Oracle.connect(user=secrets["PEN_USER"], password=secrets["PEN_PASSWORD"],
                               dsn=dsnStr)

df_ledetid = pandas_utils.pandas_from_sql('../sql/GP_BP/ledetid.sql', con)
con.close()

In [ ]:
df = df_ledetid.copy()

In [ ]:
df.head()

# Klager

In [ ]:
con = cx_Oracle.connect(user=secrets["PEN_USER"], password=secrets["PEN_PASSWORD"],
                               dsn=dsnStr)

df_klager = pandas_utils.pandas_from_sql('../sql/GP_BP/klager.sql', con)
con.close()

In [ ]:
df = df_klager.copy()
df["AR"] = df["AR"].apply(str)

In [ ]:
df_bp = df[df.STOENAD == "Barnepensjon"]
df_gjenlev = df[df.STOENAD == "Gjenlevendeytelse"]

In [ ]:
subset = {"Stadfestet", "Omgjøre", "Delvis omgjøre", "Avvist", "Opphevet og hjemvist"}
group_color = {key: theme.colors.navColors[i] for i, key in enumerate(subset)}
labels["ANTALL"] = "Antall vedtak"